# Scheirer-Ray-Hare Test

This script performs the Scheirer-Ray-Hare Test on the exercise from 'Real Statistics Using Excel' 

http://www.real-statistics.com/two-way-anova/scheirer-ray-hare-test/

The data should be in a dataframe with three columns:
1. Factor 1 (Fertilizer)
2. Factor 2 (Crop)
3. Measurement

In this example I created a file with the data from the original exercise. 
However, the code should be adaptable to any dataset provided the assumptions for the test, and the formats are the same.

For more information on this parametric test visit:

https://www.youtube.com/watch?v=N729aMGIUOk

http://rcompanion.org/handbook/F_14.html

In [1]:
# Importing the libraries. 
# pandas is necesary for the calculations
# scipy allows to estimate the p-value using the Chi-Square distribution
import pandas as pd
from scipy import stats

### Reading the data

In [2]:
data = pd.read_csv('crop_data.csv')
data.head()

,Fertilizer,Crop,Measure
0,Blend X,Wheat,123
1,Blend X,Wheat,156
2,Blend X,Wheat,112
3,Blend X,Wheat,100
4,Blend X,Wheat,168


This dataframe has three columns: Factor 1 (Fertilizer), Factor 2 (Crop), and measurement (Measure)

### Calculating the ranks

Ranks are calculated for each measurement. This step is performed on the sorted measurements regardless of the factors 

In [3]:
data['rank'] = data.Measure.sort_values().rank(numeric_only = float)
data.head()

,Fertilizer,Crop,Measure,rank
0,Blend X,Wheat,123,8.0
1,Blend X,Wheat,156,28.5
2,Blend X,Wheat,112,3.0
3,Blend X,Wheat,100,1.0
4,Blend X,Wheat,168,36.5


### Calculating the sum of the squares

In [4]:
rows = data.groupby(['Fertilizer'], as_index = False).agg({'rank':['count', 'mean', 'var']}).rename(columns={'rank':'row'})
rows.columns = ['_'.join(col) for col in rows.columns]
rows.columns = rows.columns.str.replace(r'_$',"")
rows['row_mean_rows'] = rows.row_mean.mean()
rows['sqdev'] = (rows.row_mean - rows.row_mean_rows)**2

In [5]:
cols = data.groupby(['Crop'], as_index = False).agg({'rank':['count', 'mean', 'var']}).rename(columns={'rank':'col'})
cols.columns = ['_'.join(col) for col in cols.columns]
cols.columns = cols.columns.str.replace(r'_$',"")
cols['col_mean_cols'] = cols.col_mean.mean()
cols['sqdev'] = (cols.col_mean-cols.col_mean_cols)**2

In [6]:
data_sum         = data.groupby(['Fertilizer', 'Crop'], as_index = False).agg({'rank':['count', 'mean', 'var']})
data_sum.columns = ['_'.join(col) for col in data_sum.columns]
data_sum.columns = data_sum.columns.str.replace(r'_$',"")

In [7]:
nobs_row   = rows.row_count.mean()
nobs_total = rows.row_count.sum()
nobs_col   = cols.col_count.mean()

In [8]:
Columns_SS = cols.sqdev.sum()*nobs_col
Rows_SS    = rows.sqdev.sum()*nobs_row
Within_SS  = data_sum.rank_var.sum()*(data_sum.rank_count.min()-1)
MS         = data['rank'].var()
TOTAL_SS   = MS * (nobs_total-1)
Inter_SS   = TOTAL_SS - Within_SS - Rows_SS - Columns_SS

### Calculating the H-statistics and degrees of freedom

In [9]:
H_rows = Rows_SS/MS
H_cols = Columns_SS/MS
H_int  = Inter_SS/MS

In [10]:
df_rows   = len(rows)-1
df_cols   = len(cols)-1
df_int    = df_rows*df_cols
df_total  = len(data)-1
df_within = df_total - df_int - df_cols - df_rows

### Calculating the p-values

In [11]:
p_rows  = round(1-stats.chi2.cdf(H_rows, df_rows),4)
p_cols  = round(1-stats.chi2.cdf(H_cols, df_cols),4)
p_inter = round(1-stats.chi2.cdf(H_int, df_int),4)

In [12]:
print(p_rows, p_cols, p_inter)

0.0014 0.2222 0.1478
